In [ ]:
import importlib
import requests
import time
import os


import json
from datetime import date, timedelta, datetime
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import Config as c
import Database as db

importlib.reload(db)
importlib.reload(c)

NameError: name '__file__' is not defined

In [ ]:
"""Test info"""
Reload = True
remove_minutes = True

start = date(2023, 1, 1)
end = date(2023, 1, 5)
symbol = None

In [ ]:
def get_news_from_alpaca(symbol=None, query_start=date.today(), query_end=date.today()+timedelta(days=1), headers=None):
    base_url = "https://data.alpaca.markets/v1beta1/news"
    
    params = {
        "symbols": symbol,
        "start": query_start,
        "end": query_end,
        "limit": 50,
        "sort": "desc"
    }

    page_token = None
    all_news = []
    page_number = 0
    total_items = 0
    while True:
        if page_token:
            params["page_token"] = page_token
        else:
            params.pop("page_token", None)
        print(base_url, headers, params)
        response = requests.get(base_url, headers=headers, params=params)

        if response.status_code == 429:
            print("Rate limited (429). Waiting 60 seconds...")
            time.sleep(60)
            continue
        elif response.status_code != 200:
            raise Exception(f"Alpaca API error: {response.status_code}, {response.text}")
        else:
            page_number += 1
            size = len(response.json().get("news", []))
            total_items += size
            print(f"Fetched news page {page_number} with {size} items totaling {total_items} items",end='\r', flush=True)
            

        data = response.json()
        news_page = data.get("news", [])
        all_news.extend(news_page)

        page_token = data.get("next_page_token")
        if not page_token:
            break

        time.sleep(0.1)

    return all_news


In [15]:
"""Test"""
if Reload:
    response = get_news_from_alpaca(symbol = symbol,query_start= start,query_end=end, headers={"APCA-API-KEY-ID": c.API_KEY,"APCA-API-SECRET-KEY": c.API_SECRET})


In [16]:
def strip_useless_news_data(news, remove_minutes=remove_minutes):

    stripped_news = []
    for article in news:
        new_article = article.copy()
        if "id" in new_article:
            new_article.pop("id")
        if "updated_at" in new_article:
        #    new_article["created_at"] = new_article.pop("updated_at")
            new_article.pop("updated_at")
        if "created_at" in new_article:
            new_article["date"] = datetime.fromisoformat(new_article.pop("created_at").replace("Z", "+00:00"))
            if remove_minutes:
                new_article["date"] = new_article["date"].date()
            
        if "images" in new_article:
            new_article.pop("images")
        if "url" in new_article: 
            new_article.pop("url")
        #if "content" in new_article: 
        #    new_article.pop("content")
        stripped_news.append(new_article)
    return stripped_news

In [17]:

"""Test"""


cleaner_news = strip_useless_news_data(response)
print("Cleaned news data:")
#print(cleaner_news)
index = 5
print(cleaner_news[index])
print(type(cleaner_news[index]))
print(cleaner_news[index].keys())


Cleaned news data:
{'author': 'AJ Fabino', 'content': '', 'headline': "Biden Criticizes Supreme Court's Affirmative Action Ruling", 'source': 'benzinga', 'summary': ' ', 'symbols': [], 'date': datetime.date(2023, 6, 29)}
<class 'dict'>
dict_keys(['author', 'content', 'headline', 'source', 'summary', 'symbols', 'date'])


In [18]:
def save_cached_news(cache):
    for element in cache:
        if isinstance(element, dict):
            for key, value in element.items():
                if isinstance(value, datetime):
                    element[key] = value.isoformat()
                elif isinstance(value, date):
                    element[key] = value.isoformat()
        else:
            raise ValueError("Cache must be a list of dictionaries with datetime or date values.")
    with open(c.RAW_NEWS_FILE, 'w') as f:
        json.dump(cache, f, indent=4, default=str)


def load_cached_news():
    if os.path.exists(c.RAW_NEWS_FILE):
        with open(c.RAW_NEWS_FILE, 'r') as f:
            raw = json.load(f)
        for element in raw:
            for key, value in element.items():
                if isinstance(value, str):
                    try:
                        element[key] = datetime.fromisoformat(value)
                    except ValueError:
                        try:
                            element[key] = date.fromisoformat(value)
                        except ValueError:
                            pass
        return raw
    else:     
        print(f"Cache file {c.RAW_NEWS_FILE} does not exist.")
    return {}




In [19]:
save_cached_news(cleaner_news)


In [20]:
load = db.load_cached_news()
print("Loaded cached news data:")
for i, item in enumerate(load):
    print(f"Item {i}: {item}")
    if i >= 20: 
        break
print(load[0].keys())


Loaded cached news data:
Item 0: {'author': 'Franca Quarneti', 'content': '', 'headline': "Square Enix's Final Fantasy XVI Surpasses 3 Million Copies Sold During Launch Week, But Only For PS 5", 'source': 'benzinga', 'summary': 'Square Enix Holdings Co Ltd\xa0(OTC: SQNXF), the renowned Japanese gaming company, announced impressive sales figures for its title,\xa0Final ', 'symbols': ['SONY', 'SQNXF'], 'date': datetime.datetime(2023, 6, 29, 0, 0)}
Item 1: {'author': 'Chris Katje', 'content': '', 'headline': 'Tesla Nearing Third Party App Support: Could A Tesla App Store Be Next?', 'source': 'benzinga', 'summary': 'Leading electric vehicle company Tesla Inc (NASDAQ: TSLA) announced several partnerships with leading automakers for sharing\xa0its Supercharger network. This support might lead to third-party app support and an\xa0app store coming.', 'symbols': ['F', 'GM', 'TSLA'], 'date': datetime.datetime(2023, 6, 29, 0, 0)}
Item 2: {'author': 'Charles Gross', 'content': '', 'headline': 'Nik